In [0]:
%pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 MB 151.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.7/289.7 MB 164.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import joblib
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# 1️⃣ Load saved model package
# ------------------------------------------------------------
model_package = joblib.load("phq_model.pkl")

preprocess = model_package["preprocess"]
model = model_package["model"]

# ------------------------------------------------------------
# 2️⃣ Define all features
# ------------------------------------------------------------
FEATURE_COLUMNS = preprocess.feature_names_in_

# ------------------------------------------------------------
# 3️⃣ Inference function
# ------------------------------------------------------------
def predict_phq(input_dict):
    """
    input_dict: dictionary containing all wearable biomarkers
    returns: exact PHQ score and severity category
    """
    # Ensure all expected columns exist
    for col in FEATURE_COLUMNS:
        if col not in input_dict:
            input_dict[col] = np.nan  # will be imputed

    # Remove extra keys
    input_dict = {col: input_dict[col] for col in FEATURE_COLUMNS}

    # Convert to DataFrame
    X_input = pd.DataFrame([input_dict])

    # Preprocess + predict
    X_proc = preprocess.transform(X_input)
    phq_score = model.predict(X_proc)[0]

    # Clip & round
    phq_score = float(np.clip(round(phq_score), 0, 27))

    # Severity category
    if phq_score <= 9:
        category = "Low"
    elif phq_score <= 19:
        category = "Medium"
    else:
        category = "Severe"

    return phq_score, category

# ------------------------------------------------------------
# 4️⃣ Example input with all biomarkers
# ------------------------------------------------------------
example_input = {col: 0.5 for col in FEATURE_COLUMNS}

# Modify a few features to simulate worse HR/sleep
example_input["sleep.midpoint"] = 00.9
example_input["NHRd.0204.sde"] = 0.9
example_input["IS.hri.wd"] = 0.05

# Predict
score, severity = predict_phq(example_input)

print("Predicted PHQ Score:", score)
print("Severity Level:", severity)

Predicted PHQ Score: 19.0
Severity Level: Medium


In [0]:
import mlflow
import mlflow.pyfunc
import numpy as np
import pandas as pd
import joblib

class PHQModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, preprocess, model, feature_columns):
        self.preprocess = preprocess
        self.model = model
        self.feature_columns = feature_columns

    def predict(self, context, model_input):
        # model_input: pandas DataFrame with biomarker columns
        results = []
        for _, input_dict in model_input.iterrows():
            # Ensure all expected columns exist
            input_dict = input_dict.to_dict()
            for col in self.feature_columns:
                if col not in input_dict:
                    input_dict[col] = np.nan
            # Remove extra keys
            input_dict = {col: input_dict[col] for col in self.feature_columns}
            # Convert to DataFrame
            X_input = pd.DataFrame([input_dict])
            # Preprocess + predict
            X_proc = self.preprocess.transform(X_input)
            phq_score = self.model.predict(X_proc)[0]
            phq_score = float(np.clip(round(phq_score), 0, 27))
            # Severity category
            if phq_score <= 9:
                category = "Low"
            elif phq_score <= 19:
                category = "Medium"
            else:
                category = "Severe"
            results.append({"phq_score": phq_score, "severity": category})
        return pd.DataFrame(results)

# Load model and preprocess (already in memory)
# model_package = joblib.load("phq_model.pkl")
# preprocess = model_package["preprocess"]
# model = model_package["model"]
# FEATURE_COLUMNS = preprocess.feature_names_in_

phq_model = PHQModelWrapper(preprocess, model, FEATURE_COLUMNS)

# Example input for signature
example_input = pd.DataFrame([{col: 0.5 for col in FEATURE_COLUMNS}])

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="phq_model",
        python_model=phq_model,
        input_example=example_input,
        code_path=None
    )

print("PHQ model logged to MLflow. Register in Model Registry for serving.")

/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2026/02/21 20:46:06 INFO mlflow.pyfunc: Inferring model signature from input example


PHQ model logged to MLflow. Register in Model Registry for serving.
